In [7]:
import os

import pandas as pd
import duckdb

In [8]:
for i in range(10):
    cwd = os.getcwd()
    r, c = os.path.split(cwd)
    if c == 'repo':
        print(f'new working dir: {cwd}')
        break
    os.chdir(r)

new working dir: c:\Users\aknof\Documents\GT\CSE_6250_BD4H\Project\repo


In [9]:
data_file = os.path.join('data', 'NOTEEVENTS.csv')
df = pd.read_csv(data_file)
df.head()

C:\Users\aknof\AppData\Local\Temp\ipykernel_3820\876603029.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file)


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [10]:
con = duckdb.connect()

In [11]:
df_agg = con.execute("""
select SUBJECT_ID
,string_agg(replace(TEXT, '\n', ' '), ' <> ') as PATIENT_DOC
from df
group by SUBJECT_ID
""").df()
df_agg.head()

,SUBJECT_ID,PATIENT_DOC
0,17370,Sinus tachycardia Consider inferior myocardial...
1,48641,Sinus rhythm with ventricular premature depola...
2,17588,Sinus rhythm. Consider left ventricular hypert...
3,21240,Sinus rhythm. Within normal limits. Compared t...
4,12883,Sinus rhythm Normal ECG No change from previou...


In [30]:
df_agg.loc[0, 'PATIENT_DOC'][:500]

'Sinus rhythm. Borderline left ventricular hypertrophy by voltage criteria in the limb leads. Non-diagnostic repolarization abnormalities. Compared to the previous tracing of [**2122-8-12**] no major change. TRACING #1   <> Sinus rhythm Left atrial abnormality Possible left ventricular hypertrophy Early precordial QRS transition - is nonspecific Modest nonspecific low amplitude lateral T wave changes Since previous tracing of [**2122-8-11**], probably no significant change   <> Sinus rhythm. Comp'

In [35]:
agg_file = os.path.join('data', 'consumed', 'aggregate_note_events-pipe-header.csv')
df_agg.to_csv(agg_file, index=False, sep='|', lineterminator=f'{os.linesep}', header=True)